# Preparar entorno y credenciales

## Montar drive en Colab

In [1]:
# Montar drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## GEE

In [2]:
# Instalar geemap
!pip install geemap

import geemap
import ee
ee.Authenticate()
ee.Initialize(project="ee-freddyvillota")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00


# **Parte 1: Descargar datos climáticos para modelado - Proyección del balance hídrico para 2015-2050**

## **Dibujar el bounding box de Carchi en un mapa**

In [3]:
# Cargar el polígono del nivel 1 de Ecuador (departamentos)
roi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Ecuador')) \
    .filter(ee.Filter.eq('ADM1_NAME', 'Carchi'))

# Obtener el bounding box (cuadro que lo encierra)
roi_bbox = roi.geometry().bounds()

# Crear el mapa centrado en Carchi
Map = geemap.Map()
Map.centerObject(roi, zoom=8)
Map.addLayer(roi, {'color': 'blue'}, 'Área objetivo')
Map.addLayer(roi_bbox, {'color': 'red'}, 'Bounding Box')
Map

Map(center=[0.74537837132927, -78.05993611355892], controls=(WidgetControl(options=['position', 'transparent_b…

## **Obtener las coordenadas del bounding box**

In [4]:
# Extraer coordenadas del bounding box en formato [N, W, S, E]
coords = roi_bbox.coordinates().getInfo()[0]
lats = [pt[1] for pt in coords]
lons = [pt[0] for pt in coords]

north = max(lats)
south = min(lats)
east = max(lons)
west = min(lons)

print(f"[{north:.1f}, {west:.1f}, {south:.1f}, {east:.1f}]  # [Norte, Oeste, Sur, Este]")

[1.2, -78.5, 0.4, -77.5]  # [Norte, Oeste, Sur, Este]


## **Instalar cdsapi para conectar y descargar datos climáticos futuros**

In [5]:
!pip install cdsapi

In [ ]:
# import shutil

# # Copiar .cdsapirc al directorio que espera cdsapi
# ruta_drive = '/content/drive/MyDrive/secrets/.cdsapirc'
# ruta_destino = '/root/.cdsapirc'
# shutil.copyfile(ruta_drive, ruta_destino)

### **Generar código en la página de CDS**

✅ Usar el API actualizado desde la web oficial

1. 🔧 Paso 1: Acceder a la interfaz gráfica del CDS para CMIP6
👉 https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=form

2. 🛠 Paso 2: Llenar el formulario. Por ejemplo:
Temporal resolution: Monthly
  - Variable: Precipitation
  - Experiment: SSP2-4.5
  - Model: el de preferencia (ej. CNRM-ESM2-1)
  - Ensemble member: r1i1p1f1
  - Period: 2025-01 to 2050-12
  - Region: la que se defina según el área de interés [1.5, -79.0, 0.2, -76.0])
  - Marca "NetCDF" como formato

3. 🧾 Paso 3: Hacer clic en "Show API request"
Esto generará el código exacto con la sintaxis actualizada y válida.

### **Exportar cada una de las variables climáticas con el código generado en CDS**

In [ ]:
# import cdsapi
# # Variables a descargar:
#   # 1. orthward_wind
#   # 2. precipitation
#   # 3. near_surface_air_temperature
#   # 4. surface_downwelling_shortwave_radiation
#   # 5. eastward_wind

# dataset = "projections-cmip6"
# request = {
#     "temporal_resolution": "monthly",
#     "experiment": "ssp2_4_5",
#     "variable": "eastward_wind",
#     "model": "cnrm_esm2_1",
#     "year": [
#         "2015", "2016", "2017",
#         "2018", "2019", "2020",
#         "2021", "2022", "2023",
#         "2024", "2025", "2026",
#         "2027", "2028", "2029",
#         "2030", "2031", "2032",
#         "2033", "2034", "2035",
#         "2036", "2037", "2038",
#         "2039", "2040", "2041",
#         "2042", "2043", "2044",
#         "2045", "2046", "2047",
#         "2048", "2049"
#     ],
#     "month": [
#         "01", "02", "03",
#         "04", "05", "06",
#         "07", "08", "09",
#         "10", "11", "12"
#     ],
#     'area': [22.7, -105.7, 19.0, -101.5],
# }

# client = cdsapi.Client()
# client.retrieve(dataset, request).download("eastward_wind.zip")

**Hay que esperar hasta que el archivo este disponible en la plataforma de CDS. Es muy tardado, dependiendo de la extensión del área de estudio, períodos solicitados y la cantidad de variables.**

# **Parte 2: Procesar archivos descargados**

Primero se descargan los archivos zip desde la plataforma CDS. Después se cargan al drive para continuar con el procesamiento.

### **Descomprimir el archivo zip**

In [ ]:
# Extraer para precipitation
import zipfile

zip_path_radiation = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/surface_downwelling_shortwave_radiation-Carchi.zip'
extract_path_radiation = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

with zipfile.ZipFile(zip_path_radiation, 'r') as zip_ref:
    zip_ref.extractall(extract_path_radiation)


# Extraer para shortwave radiation
import zipfile

zip_path_orthward_wind = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/orthward_wind-Carchi.zip'
extract_path_orthward_wind = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

with zipfile.ZipFile(zip_path_orthward_wind, 'r') as zip_ref:
    zip_ref.extractall(extract_path_orthward_wind)

# Extraer para northward wind
import zipfile

zip_path_precipitation = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/precipitation-Carchi.zip'
extract_path_precipitation = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

with zipfile.ZipFile(zip_path_precipitation, 'r') as zip_ref:
    zip_ref.extractall(extract_path_precipitation)

# Extraer para near-surface air temperature
import zipfile

zip_path_temperature = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/near_surface_air_temperature-Carchi.zip'
extract_path_temperature = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

with zipfile.ZipFile(zip_path_temperature, 'r') as zip_ref:
    zip_ref.extractall(extract_path_temperature)

# Extraer para eastward wind
import zipfile

zip_path_eastward_wind = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/eastward_wind-Carchi.zip'
extract_path_eastward_wind = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

with zipfile.ZipFile(zip_path_eastward_wind, 'r') as zip_ref:
    zip_ref.extractall(extract_path_eastward_wind)

**Verifica el nombre del archivo .nc**

In [ ]:
import os

# Lista de archivos extraídos
os.listdir(extract_path_radiation)

### **Procesar archivo NetCDF a DataFrame**

In [6]:
import os
import xarray as xr
import pandas as pd
from functools import reduce

# Ruta base
base_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi'

# Mapeo carpeta → nombre de variable
var_map = {
    'cmip6_precip_data': 'precip_mm',
    'cmip6_temp_data': 'temp_c',
    'cmip6_eastwardwind_data': 'wind_u',
    'cmip6_northwind_data': 'wind_v',
    'cmip6_radiation_data': 'solar_rad'
}

df_list = []

for folder, col_name in var_map.items():
    folder_path = os.path.join(base_path, folder)
    nc_file = [f for f in os.listdir(folder_path) if f.endswith('.nc')][0]
    full_path = os.path.join(folder_path, nc_file)

    ds = xr.open_dataset(full_path)

    # Excluir variables como time_bounds
    var_candidates = [v for v in ds.data_vars if 'time' in ds[v].dims and 'bounds' not in v.lower()]
    var_name = var_candidates[0]

    da = ds[var_name]

    # Promediar sobre todas las dimensiones menos el tiempo
    dims_to_avg = [d for d in da.dims if d != 'time']
    if dims_to_avg:
        da = da.mean(dim=dims_to_avg)

    # Extraer tiempo y valores
    time = da['time'].values
    values = da.values

    # Verificación de longitudes
    if len(time) != len(values):
        print(f"❗ Tamaños distintos para {col_name}: fechas={len(time)}, valores={len(values)}. Saltando esta variable.")
        continue

    df_var = pd.DataFrame({
        'date': pd.to_datetime(time),
        col_name: values
    })
    df_var['year'] = df_var['date'].dt.year
    df_var['month'] = df_var['date'].dt.month

    df_list.append(df_var)

# Fusionar todos los DataFrames
df_cmip6 = reduce(lambda left, right: pd.merge(left, right, on=['date', 'year', 'month']), df_list)
df_cmip6.head()

,date,precip_mm,year,month,temp_c,wind_u,wind_v,solar_rad
0,2015-01-16 12:00:00,0.000109,2015,1,291.423828,-9.422890,0.641277,195.437805
1,2015-02-15 00:00:00,0.000185,2015,2,291.435791,-7.968413,0.566743,193.973434
2,2015-03-16 12:00:00,0.000312,2015,3,291.304688,-4.723524,0.570539,151.706879
3,2015-04-16 00:00:00,0.000331,2015,4,291.405762,-2.849622,0.169271,150.485321
4,2015-05-16 12:00:00,0.000273,2015,5,290.719208,-1.718866,0.157390,154.813721


### **Cambiar unidades a las variables climáticas**

**Detalles técnicos confirmados**

1. Temperatura: K a °C ✔️

2. Precipitación: kg/m²/s = mm/s, y multiplicar por 86400 s/día es lo correcto ✔️

3. Radiación solar:

  - W/m² × segundos del mes (3600 × 24 × n días) → J/m² ÷1e6 → MJ/m² ✔️

  - Coincide con el tratamiento de ERA5 (J/m² → MJ/m²) ✔️

In [7]:
# Aplicar conversiones sobre df_cmip6 antes de predicción
df_cmip6['temp_c'] = df_cmip6['temp_c'] - 273.15  # K → °C
df_cmip6['precip_mm'] = df_cmip6['precip_mm'] * 86400  # kg/m²/s → mm/día

# El valor es un promedio mensual en W/m², convertir a MJ/m²/mes:
# MJ/m² = W/m² * 3600 s/h * 24 h/d * días/mes / 1e6
dias_del_mes = df_cmip6['date'].dt.days_in_month
df_cmip6['solar_rad'] = df_cmip6['solar_rad'] * 3600 * 24 * dias_del_mes / 1e6  # → MJ/m²/mes
df_cmip6.head()

,date,precip_mm,year,month,temp_c,wind_u,wind_v,solar_rad
0,2015-01-16 12:00:00,9.386863,2015,1,18.273834,-9.422890,0.641277,523.460668
1,2015-02-15 00:00:00,15.993876,2015,2,18.285797,-7.968413,0.566743,469.260540
2,2015-03-16 12:00:00,26.977810,2015,3,18.154694,-4.723524,0.570539,406.331694
3,2015-04-16 00:00:00,28.607992,2015,4,18.255768,-2.849622,0.169271,390.057930
4,2015-05-16 12:00:00,23.624590,2015,5,17.569214,-1.718866,0.157390,414.653055


## **Exportar a drive en formato a excel directo**

In [ ]:
# # Definir ruta y nombre del archivo
# export_path = '/content/drive/MyDrive/MIAA/Clases/Herramientas IA/Prácticas/balance_hidrico/data/cmip6/Carchi/df_cmip6.csv'

# # Guardar como CSV
# df_cmip6.to_csv(export_path, index=False)

# print("✅ Archivo exportado correctamente a:")
# print(export_path)

# **Trabajar con SQL en Supabase**

En el dashboard de Supabase (https://supabase.com/dashboard/project/lyegpknqtizanhbzznnj):

1. Seleccionar el proyecto.
2. Ir a la pestaña Database > SQL Editor.
3. Ejecutar el script para crear la tabla (ajustar a las columnas reales del  DataFrame). Por ejemplo:


```
sql

create table balance_proyectado (
  date date,
  year int,
  month int,
  temp_c double precision,
  precip_mm double precision,
  wind_u double precision,
  wind_v double precision,
  solar_rad double precision
);

```

## **Subir datos al SQL a partir de un csv**

- En el menú izquierdo, hacer clic en Table Editor → Import Data.
- Elegir el archivo .csv.
- Supabase detectará automáticamente los nombres y tipos de columnas.
- También se selecciona a la tabla que se quiere agregar los datos.
- Hacer clic en Importar.

## **Leer la tabla balance_proyectado en Supabase mediante la REST API de Supabase**

Obtener la siguiente información del proyecto Supabase. Dar clic en connect después en App frameworks y obtener:

1. URL del proyecto Supabase (NEXT_PUBLIC_SUPABASE_URL)
2. API Key tipo anon (NEXT_PUBLIC_SUPABASE_ANON_KEY)
3. El nombre exacto de la tabla (balance_proyectado)
4. Importante guardar estas claves en secrets de colab

In [8]:
import requests
import pandas as pd
from google.colab import userdata

# 1. Leer secretos
url = userdata.get("SUPABASE_URL")
api_key = userdata.get("SUPABASE_API_KEY")

# 2. Endpoint para la tabla
endpoint = f"{url}/rest/v1/balance_proyectado"

# 3. Headers de autenticación
headers = {
    "apikey": api_key,
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# 4. Hacer la consulta GET
response = requests.get(endpoint, headers=headers, params={"select": "*"})

# 5. Convertir a DataFrame
if response.status_code == 200:
    data = response.json()
    df_sql = pd.DataFrame(data)
    print("✅ Datos cargados correctamente desde Supabase vía REST.")
    display(df_sql.head())
else:
    print(f"❌ Error {response.status_code}: {response.text}")

✅ Datos cargados correctamente desde Supabase vía REST.


,date,year,month,temp_c,precip_mm,wind_u,wind_v,solar_rad
0,2015-01-16,2015,1,291.42383,0.000109,-9.422890,0.641277,195.43780
1,2015-02-15,2015,2,291.43580,0.000185,-7.968413,0.566743,193.97343
2,2015-03-16,2015,3,291.30470,0.000312,-4.723524,0.570539,151.70688
3,2015-04-16,2015,4,291.40576,0.000331,-2.849622,0.169271,150.48532
4,2015-05-16,2015,5,290.71920,0.000273,-1.718866,0.157390,154.81372


## **Explorar dataset visualmente**

In [11]:
!pip install ydata-profiling --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.4 MB/s eta 0:00:00


In [13]:
from ydata_profiling import ProfileReport

# Asegura que la fecha esté en formato correcto
df_sql['date'] = pd.to_datetime(df_sql['date'])

# Generar el perfil
profile = ProfileReport(df_sql, title="Análisis Exploratorio del Balance Hídrico", explorative=True)

# Mostrar en el notebook
profile.to_notebook_iframe()


Output hidden; open in https://colab.research.google.com to view.